In [ ]:
from brainiak.isc import isc,bootstrap_isc,compute_summary_statistic,isfc,squareform_isfc
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from isc_tools import *

In [ ]:
froot = '/data00/layerfMRI/'
datafld = froot + 'Github_repo/layerfMRI/analyses/dual_ISC/TC_4_lorenzo/'

In [ ]:
summary_log = import_summary_log(froot)
movie_length = extract_movie_length(summary_log)

In [ ]:
## Parameters corresponding to data to be analyzed
nbins = 6
thr = 100
method = 'M_OR_S'

input_file = datafld + f'{method}_JU_time_courses_thr{thr}_bin{nbins}.csv'

In [ ]:
df = import_timecourse_dataframe(
    input_file,
    check_movie_length = True, movie_length = movie_length
)

cdf  =concatenate_movies(df,average_same_movie=False)
# cdf1 =concatenate_movies(df,average_same_movie=True)

In [ ]:
## Exclude subject 11
cdf = exclude_subjects(cdf,['sub_11'])

In [ ]:
## Extra layer of safety
cdf = doublecheck_same_lenght(cdf)

In [ ]:
cdf = compute_isc(cdf)

In [ ]:
plot_export_isc(cdf,input_file,method)

In [ ]:
isc_column = np.array([])

subJUcdf = cdf.loc[lambda d : (d['JU']==13)|(d['JU']==35)|(d['JU']==200)|(d['JU']==91)]
subJUcdf = subJUcdf.sort_values(['JU','D_bins'])
isfc_data = {}
isfc_mean_data = {}
nsub = subJUcdf['sub'].nunique()

sel_JUs = {'BA44':13, 'PFt':35, 'Vis':200,'BA6':91}

nJU = len(sel_JUs)

for c,contrast_df in subJUcdf.groupby('contrast'):

    iscarr = np.array(list(contrast_df
     .groupby('sub')
     .apply(lambda d: np.array(list(d['tc_concatenated'])).T)
    ))

    iscarr = np.moveaxis(iscarr,0,2)
    print(iscarr.shape)

    isc_data =  isc(iscarr)
    raw_isfc = isfc(iscarr)[0]
    isfc_data[c] = np.zeros((nsub,int(nbins*nJU*(nbins*nJU-1)/2)))
    for r,sub_isfc in enumerate(raw_isfc):
        isfc_data[c][r,:] = np.arctanh(sub_isfc)    
    isfc_mean_data[c] = np.average(isfc_data[c],axis = 0)
    
    
    isc_column = np.append(isc_column,isc_data.ravel())

subJUcdf['isc'] = np.arctanh(isc_column)


In [ ]:
from scipy.spatial.distance import squareform

In [ ]:
from scipy.stats import ttest_1samp,ttest_rel
from statsmodels.stats.multitest import fdrcorrection

In [ ]:
for c in ['Motion','Scrambled']:
    M = 0.2

    fig,axarr = plt.subplots(nrows = 1, ncols=1,#+1, gridspec_kw={'width_ratios':[10]*nJU+[1]})#,
                             sharex=True, sharey=True, figsize = (6,6))


    z = isfc_data[c]
    t,p = ttest_1samp(z, popmean = 0, axis=0)
    pmask, pcorr = fdrcorrection(p,alpha = 0.05, is_sorted=False)

    
    z_matrix = squareform(np.average(z,axis=0)*(pmask))
    cm = axarr.imshow(z_matrix,
                        cmap = 'seismic', vmax=M,vmin=-M)
                
    fout = datafld.replace('TC_4_lorenzo/','ISFC/results_lorenzo/new_ISFC_results/') + '{}_full_thr{}_bin{}.txt'.format(c,thr,nbins)
#     fout = '/data00/layerfMRI/Github_repo/layerfMRI/analyses/dual_ISC/4figures/S2_comparison_ISFC_FC_matrices/{}_full_thr{}_bin{}.txt'.format(c,thr,nbins)
    np.savetxt(fout,z_matrix)
    
#     plt.colorbar(cm,cax = axarr[1,3])

    
    plt.suptitle(c)


In [ ]:
sel_JUs = {'BA44':13, 'PFt':35, 'Vis':200, 'BA6' : 91}
list_JUs = list(sel_JUs)

nJU = len(sel_JUs)

for c in isfc_mean_data:
#     M = 0.25
    
    
    z = isfc_data[c]
    t,p = ttest_1samp(z, popmean = 0, axis=0)
    pmask, pcorr = fdrcorrection(p,alpha = 0.05)

    full_matrix = squareform(t*(pmask))
    fout = '/data00/layerfMRI/Github_repo/layerfMRI/analyses/dual_ISC/4figures/S2_comparison_ISFC_FC_matrices/{}_full_thr{}_bin{}.txt'.format(c,thr,nbins)
    np.savetxt(fout,full_matrix)    
    print(fout)
    
    
    M = 25
#     M = np.max(np.abs(t))
#     print(M)

    fig,axarr = plt.subplots(nrows = nJU, ncols=nJU,#+1, gridspec_kw={'width_ratios':[10]*nJU+[1]})#,
                             sharex=True, sharey=True, figsize = (6,6))

    for ix in range(nJU):

        # Labels JU
        axarr[ix,-1].yaxis.set_label_position("right")
        axarr[ix,-1].set_ylabel(list_JUs[ix], rotation = 270, labelpad=14)
        axarr[0,ix].xaxis.set_label_position("top")
        axarr[0,ix].set_xlabel(list_JUs[ix], labelpad=4)
        # Labels bins
        axarr[ix,0].set_yticks(np.arange(0,nbins,5))
        axarr[ix,0].set_yticklabels(np.arange(0,nbins,5)+1)
        axarr[-1,ix].set_xticks(np.arange(0,nbins,5))
        axarr[-1,ix].set_xticklabels(np.arange(0,nbins,5)+1)
        axarr[ix,0].minorticks_on()
        
        for iy in range(nJU):
            

            z_matrix = squareform(t*(pmask))[nbins*iy:nbins*(iy+1),nbins*ix:nbins*(ix+1)]
            cm = axarr[iy,ix].imshow(z_matrix,
                                cmap = 'seismic', vmax=M,vmin=-M)


            if (list_JUs[iy]=='PFt')&((list_JUs[ix]=='Vis')|(list_JUs[ix]=='BA44')):
    #             fout = datafld.replace('TC_4_lorenzo/','ISFC/results_lorenzo/new_ISFC_results/') + 't-values_{}_{}-{}_thr{}_bin{}.txt'.format(c,list_JUs[iy],list_JUs[ix],thr,nbins)
                fout = '/data00/layerfMRI/Github_repo/layerfMRI/analyses/dual_ISC/4figures/fig_4_ISFC_matrices/t-values_{}_{}-{}_thr{}_bin{}.txt'.format(c,list_JUs[iy],list_JUs[ix],thr,nbins)
                np.savetxt(fout,z_matrix)
                print(fout)
            
#     plt.colorbar(cm,cax = axarr[1,3])
    
    axarr[1,0].set_ylabel('# bin')
    axarr[-1,1].set_xlabel('# bin')
    
    plt.suptitle(c)


In [ ]:
fig,ax = plt.subplots(figsize = (0.3,6))
plt.colorbar(cm,cax = ax,orientation = 'vertical')
ax.set_ylabel('mean Z transformed ISFC')
plt.show()

In [ ]:
zm = isfc_data['Motion']
zs = isfc_data['Scrambled']

t,p = ttest_rel(zm,zs, axis=0)
pmask, pcorr = fdrcorrection(p,alpha = 0.05)

full_matrix = squareform(t*(pmask))
fout = '/data00/layerfMRI/Github_repo/layerfMRI/analyses/dual_ISC/4figures/S2_comparison_ISFC_FC_matrices/Motion-Scrambled_full_thr{}_bin{}.txt'.format(thr,nbins)
np.savetxt(fout,full_matrix)   
print(fout)

M = 7
# M = np.max(np.abs(t))
# M = np.max(np.abs(zm-zs))

fig,axarr = plt.subplots(nrows = nJU, ncols=nJU,#+1, gridspec_kw={'width_ratios':[10]*nJU+[1]})#,
                         sharex=True, sharey=True, figsize = (6,6))

for ix in range(nJU):

    # Labels JU
    axarr[ix,-1].yaxis.set_label_position("right")
    axarr[ix,-1].set_ylabel(list(sel_JUs)[ix], rotation = 270, labelpad=14)
    axarr[0,ix].xaxis.set_label_position("top")
    axarr[0,ix].set_xlabel(list(sel_JUs)[ix], labelpad=4)
    # Labels bins
    axarr[ix,0].set_yticks(np.arange(0,nbins,5))
    axarr[ix,0].set_yticklabels(np.arange(0,nbins,5)+1)
    axarr[-1,ix].set_xticks(np.arange(0,nbins,5))
    axarr[-1,ix].set_xticklabels(np.arange(0,nbins,5)+1)
    axarr[ix,0].minorticks_on()
        
    for iy in range(nJU):



        z_matrix = squareform(t*(pmask))[nbins*iy:nbins*(iy+1),nbins*ix:nbins*(ix+1)]
#         z_matrix = squareform(np.average(zm-zs,axis=0)*(pmask))[nbins*iy:nbins*(iy+1),nbins*ix:nbins*(ix+1)]
        cm = axarr[iy,ix].imshow(z_matrix,
                            cmap = 'seismic', vmax=M,vmin=-M)


        
        if (list_JUs[iy]=='PFt')&((list_JUs[ix]=='Vis')|(list_JUs[ix]=='BA44')):
            fout = '/data00/layerfMRI/Github_repo/layerfMRI/analyses/dual_ISC/4figures/fig_4_ISFC_matrices/t-values_Motion-Scrambled_{}-{}_thr{}_bin{}.txt'.format(list_JUs[iy],list_JUs[ix],thr,nbins)       
#             fout = datafld.replace('TC_4_lorenzo/','ISFC/results_lorenzo/new_ISFC_results/') + 't-values_Motion-Scrambled_{}-{}_thr{}_bin{}.txt'.format(list_JUs[iy],list_JUs[ix],thr,nbins)
            np.savetxt(fout,z_matrix)
            print(fout)
        
#     plt.colorbar(cm,cax = axarr[1,3])
axarr[1,0].set_ylabel('# bin')
axarr[-1,1].set_xlabel('# bin')
    

plt.suptitle('Motion - Scrambled')

In [ ]:
fig,ax = plt.subplots(figsize = (0.3,6))
plt.colorbar(cm,cax = ax,orientation = 'vertical')
ax.set_ylabel('mean Z transformed ISFC')
plt.show()

In [ ]:
M

## PPI-like fc

In [ ]:
isc_column = np.array([])

subJUcdf = cdf.loc[lambda d : (d['JU']==13)|(d['JU']==35)|(d['JU']==200)|(d['JU']==91)]
subJUcdf = subJUcdf.sort_values(['JU','D_bins'])
isfc_data = {}
isfc_mean_data = {}
nsub = subJUcdf['sub'].nunique()

sel_JUs = {'BA44':13, 'PFt':35, 'Vis':200, 'BA6':91}

nJU = len(sel_JUs)

for c,contrast_df in subJUcdf.groupby('contrast'):

    iscarr = np.array(list(contrast_df
     .groupby('sub')
     .apply(lambda d: np.array(list(d['tc_concatenated'])).T)
    ))

    iscarr = np.moveaxis(iscarr,0,2)
    
    raw_isfc = generic_isfc(iscarr,intersubject=False)
    isfc_data[c] = np.zeros((nsub,int(nbins*nJU*(nbins*nJU-1)/2)))
    for r,sub_isfc in enumerate(raw_isfc):
        isfc_data[c][r,:] = np.arctanh(sub_isfc)    
    isfc_mean_data[c] = np.average(isfc_data[c],axis = 0)
    
    
    isc_column = np.append(isc_column,isc_data.ravel())

subJUcdf['isc'] = np.arctanh(isc_column)


In [ ]:
sel_JUs = {'BA44':13, 'PFt':35, 'Vis':200, 'BA6':91}
list_JUs = list(sel_JUs)

nJU = len(sel_JUs)

for c in isfc_mean_data:
    
    
    z = isfc_data[c]
    t,p = ttest_1samp(z, popmean = 0, axis=0)
    pmask, pcorr = fdrcorrection(p,alpha = 0.05)
    M = np.max(np.abs(t))
    print(M)

    full_matrix = squareform(t*(pmask))
    fout = '/data00/layerfMRI/Github_repo/layerfMRI/analyses/dual_ISC/4figures/S2_comparison_ISFC_FC_matrices/WSFC_{}_full_thr{}_bin{}.txt'.format(c,thr,nbins)
    np.savetxt(fout,full_matrix)   
    print(fout)
    
    

    fig,axarr = plt.subplots(nrows = nJU, ncols=nJU,#+1, gridspec_kw={'width_ratios':[10]*nJU+[1]})#,
                             sharex=True, sharey=True, figsize = (6,6))

    for ix in range(nJU):

        # Labels JU
        axarr[ix,-1].yaxis.set_label_position("right")
        axarr[ix,-1].set_ylabel(list_JUs[ix], rotation = 270, labelpad=14)
        axarr[0,ix].xaxis.set_label_position("top")
        axarr[0,ix].set_xlabel(list_JUs[ix], labelpad=4)
        # Labels bins
        axarr[ix,0].set_yticks(np.arange(0,nbins,5))
        axarr[ix,0].set_yticklabels(np.arange(0,nbins,5)+1)
        axarr[-1,ix].set_xticks(np.arange(0,nbins,5))
        axarr[-1,ix].set_xticklabels(np.arange(0,nbins,5)+1)
        axarr[ix,0].minorticks_on()
        
        for iy in range(nJU):
            

            z_matrix = squareform(t*(pmask))[nbins*iy:nbins*(iy+1),nbins*ix:nbins*(ix+1)]
            cm = axarr[iy,ix].imshow(z_matrix,
                                cmap = 'seismic', vmax=M,vmin=-M)

    
            if (list_JUs[iy]=='PFt')&((list_JUs[ix]=='Vis')|(list_JUs[ix]=='BA44')):
                fout = '/data00/layerfMRI/Github_repo/layerfMRI/analyses/dual_ISC/4figures/fig_4_ISFC_matrices/WSFC_t-values_{}_{}-{}_thr{}_bin{}.txt'.format(c,list_JUs[iy],list_JUs[ix],thr,nbins)       
#                 fout = datafld.replace('TC_4_lorenzo/','ISFC/results_lorenzo/new_ISFC_results/') + 'WSFC_t-values_{}_{}-{}_thr{}_bin{}.txt'.format(c,list_JUs[iy],list_JUs[ix],thr,nbins)
                np.savetxt(fout,z_matrix)
                print(fout)
            
#     plt.colorbar(cm,cax = axarr[1,3])
    
    axarr[1,0].set_ylabel('# bin')
    axarr[-1,1].set_xlabel('# bin')
    
    plt.suptitle(c)


In [ ]:
zm = isfc_data['Motion']
zs = isfc_data['Scrambled']

t,p = ttest_rel(zm,zs, axis=0)
pmask, pcorr = fdrcorrection(p,alpha = 0.05)


full_matrix = squareform(t*(pmask))
fout = '/data00/layerfMRI/Github_repo/layerfMRI/analyses/dual_ISC/4figures/S2_comparison_ISFC_FC_matrices/WSFC_Motion-Scrambled_full_thr{}_bin{}.txt'.format(thr,nbins)
np.savetxt(fout,full_matrix)   
print(fout)

fig,axarr = plt.subplots(nrows = nJU, ncols=nJU,#+1, gridspec_kw={'width_ratios':[10]*nJU+[1]})#,
                         sharex=True, sharey=True, figsize = (6,6))

for ix in range(nJU):

    # Labels JU
    axarr[ix,-1].yaxis.set_label_position("right")
    axarr[ix,-1].set_ylabel(list(sel_JUs)[ix], rotation = 270, labelpad=14)
    axarr[0,ix].xaxis.set_label_position("top")
    axarr[0,ix].set_xlabel(list(sel_JUs)[ix], labelpad=4)
    # Labels bins
    axarr[ix,0].set_yticks(np.arange(0,nbins,5))
    axarr[ix,0].set_yticklabels(np.arange(0,nbins,5)+1)
    axarr[-1,ix].set_xticks(np.arange(0,nbins,5))
    axarr[-1,ix].set_xticklabels(np.arange(0,nbins,5)+1)
    axarr[ix,0].minorticks_on()
        
    for iy in range(nJU):



        z_matrix = squareform(t*(pmask))[nbins*iy:nbins*(iy+1),nbins*ix:nbins*(ix+1)]
        cm = axarr[iy,ix].imshow(z_matrix,
                            cmap = 'seismic', vmax=M,vmin=-M)


        
        if (list_JUs[iy]=='PFt')&((list_JUs[ix]=='Vis')|(list_JUs[ix]=='BA44')):
            fout = '/data00/layerfMRI/Github_repo/layerfMRI/analyses/dual_ISC/4figures/fig_4_ISFC_matrices/WSFC_t-values_Motion-Scrambled_{}-{}_thr{}_bin{}.txt'.format(list_JUs[iy],list_JUs[ix],thr,nbins)          
#                 fout = datafld.replace('TC_4_lorenzo/','ISFC/results_lorenzo/new_ISFC_results/') + 'WSFC_t-values_Motion-Scrambled_{}-{}_thr{}_bin{}.txt'.format(list_JUs[iy],list_JUs[ix],thr,nbins)
            np.savetxt(fout,z_matrix)
            print(fout)
        
#     plt.colorbar(cm,cax = axarr[1,3])
axarr[1,0].set_ylabel('# bin')
axarr[-1,1].set_xlabel('# bin')
    

plt.suptitle('Motion - Scrambled')


In [ ]:
fig,ax = plt.subplots(figsize = (0.3,6))
plt.colorbar(cm,cax = ax,orientation = 'vertical', ticks = np.arange(-M,M+0.1,0.5))
ax.set_ylabel('mean Z transformed ISFC')
plt.show()